## Laboratorio 2: Redes Neuronales
### Miguel Valle - 17102

El objetivo de este laboratorio fue implementar un algoritmo de redes neuronales para la creación de un modelo de predicción basada en la clasificación de imágenes de ropa según el tipo de prenda al que pertenece

### Funciones 

In [1]:
#Se importan las librerias que se van a utilizar
import numpy as np
import mnist_reader
import pandas as pd
from functools import reduce
from scipy import optimize as op

In [2]:
#Metodo para preparar las matrices de thetas y que estas se 
#puedan operar con los datos
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)
    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

In [17]:
#Funcion de sigmoide para la evaluacion de los datos
def sigmoid(v):
    s = [(1 / (1 + np.exp(-i))) for i in v]
    return np.asarray(s).reshape(v.shape)

In [4]:
#Funcion de foward propagation para calcular la salida de 
#cada capa de neuronas
def feed_forward(thetas, X):
    a = [np.asarray(X)]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                        a[i]
                    )), thetas[i].T
                )
            )            
        )
    return a

In [5]:
#Funcion de costo para la salida de cada capa de neuronas
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

In [18]:
#Funcion de back propagation para calcular el gradiente
def cost_function_jacobian(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    f = feed_forward(thetas, X)
    d = [*range(layers - 1), f[-1] - Y]
    for i in range(layers - 2, 0, -1):
        d[i] = (d[i + 1] @ np.delete(thetas[i], 0, 1)) * (f[i] * (1 - f[i]))
    Deltas = []
    for i in range(layers - 1):
        Deltas.append(
            (d[i + 1].T @ np.hstack((
                np.ones(len(f[i])).reshape(len(f[i]), 1), f[i]
            ))) / m
        )
    Deltas = np.asarray(Deltas)
    return flatten_list_of_arrays(
        Deltas
    )

In [7]:
#Funcion para preparar los datos para Scipy
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

### Preparacion del Modelo

A continuación se cargan los datos de enternamiento de los archivos de entrenamiento y de prueba. Dado que ambos contienen los valores de las matrices de las imágenes y el número que indica la categoría a la que pertenece la prenda en la imágen, el modelo se entrena utilizando todos los datos ya que mientras más datos se utilizan mejor.

### Nota: Debido a que el tamaño del archivo fashion-mnist_train.csv es mayor a limite de 100mb que tiene github, no se pudo subir sin comprimirlo antes, por lo cual si se desea probar el codigo se debe clonar el repositorio y extraer el archivo fashion-mnist_train.zip

In [8]:
#Se leen los datos de prueba
dataset_test = pd.read_csv('fashion-mnist_test.csv')

#Se leen los datos de entrenamiento
dataset_train = pd.read_csv('fashion-mnist_train.csv')

#Se obtienen los datos de las imagenes y la clasificacion
#de tipo de prenda para cada imagen
x1 = dataset_train.iloc[:, 1:] / 1000.0
m1, n1 = x1.shape
x2 = dataset_test.iloc[:, 1:] / 1000.0
m2, n2 = x2.shape
y1 = np.asarray(dataset_train.iloc[:, 0])
y1 = y1.reshape(m1, 1)
y2 = np.asarray(dataset_test.iloc[:, 0])
y2 = y2.reshape(m2, 1)
#Se juntan los datos de test y train para tener mayor
#cantidad de datos para entrenar el modelo
X = np.vstack((
    x1,
    x2
))
m, n = X.shape
#Se juntan los datos de clasificacion de test y train correspondientes
y = np.vstack((
    y1,
    y2
))
y = y.reshape(m, 1)

#Se prepara la matriz de clasificacion de prenda
Y = (y == np.array(range(10))).astype(int)

Se define la estructura de la red neuronal, se preparan los theta iniciales de forma aleatoria con la forma necesaria para luego iniciar el proceso de entrenamiento del modelo

#### Nota: No se recomiendo ejecutar este bloque de codigo ya que el entrenamiento puede durar mucho tiempo, por lo cual se recomiendo seguir al bloque en el cual se prueba la efectividad del modelo producido.

In [26]:
#Se define la estructura de la red neuronal
red = np.array([
    n,
    130,
    10
])
#Se obtienen los shapes de los diferentes matrices de theta
#para cada capa
theta_shapes = np.hstack((
    red[1:].reshape(len(red) - 1, 1),
    (red[:-1] + 1).reshape(len(red) - 1, 1)
))
#Se generan valores aleatorio iniciales para los thetas
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape) 
    for theta_shape in theta_shapes
])
#Se inicia el modelo de la red neuronal
result = op.minimize(
    fun=cost_function,
    x0=flat_thetas,
    args=(theta_shapes,X,Y),
    method='L-BFGS-B',
    jac=cost_function_jacobian,
    options={'disp': True, 'maxiter': 3000}
)

print(result)
print(result.x)

Lectura de mnist_train
Lectura de mnist_test
(70000, 784)
(70000, 10)
(2, 2)


c:\users\hecto\documents\uvg 2020\inteligencia artificial\lab 2\lab2-ia\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
c:\users\hecto\documents\uvg 2020\inteligencia artificial\lab 2\lab2-ia\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


      fun: nan
 hess_inv: <103360x103360 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 9.61879487e-06,  2.64313123e-13,  2.88470994e-10, ...,
       -8.07560078e-05, -8.07504521e-05, -8.07510232e-05])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 10741
      nit: 3000
   status: 1
  success: False
        x: array([-2.30253597,  0.32329947,  0.61515723, ..., -0.8653714 ,
       -0.8958914 ,  0.09391501])
[-2.30253597  0.32329947  0.61515723 ... -0.8653714  -0.8958914
  0.09391501]


In [27]:
#Se almacenan los theta resultantes en un archivo de texto 
np.savetxt('thetas.txt', result.x)

### Prueba de Modelo

Por si solo se desea probar el modelo producido se carga de nuevo el archivo de datos para pruebas, y se cargan los valores de theta producidos por el entrenamiento del modelo anteriormente.

In [9]:
dataset_test = pd.read_csv('fashion-mnist_test.csv')
x_test = dataset_test.iloc[:, 1:] / 1000.0
y_test = np.asarray(dataset_test.iloc[:, 0]).reshape(x_test.shape[0], 1)

In [11]:
red = np.array([
    n,
    130,
    10
])
theta_shapes = np.hstack((
    red[1:].reshape(len(red) - 1, 1),
    (red[:-1] + 1).reshape(len(red) - 1, 1)
))
thetas_r = np.loadtxt("thetas.txt")
thetas = inflate_matrixes(thetas_r, theta_shapes)
f = feed_forward(thetas, x_test)
eva = np.argmax(f[-1], axis = 1)

In [19]:
exito = 0
for i in range(len(f[-1])):
    if (eva[i] == y_test[i][0]):
        exito += 1
print("Aciertos: ", exito * 100/len(f[-1])," %")

Aciertos:  90.3  %


## Conclusiones

Como se puede observar en el bloque de arriba, se logró crear un modelo a partir del dataset proveído, de tal manera que al utilizarlo para realizar la clasificación de un dataset de imágenes de prueba, esta clasificación acertó en el 90.3 % de las imágenes, lo cual se puede considerar como un buen modelo.